<a href="https://colab.research.google.com/github/PSCostaM/TA1_CC82_TopicosCS/blob/master/TP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.2

In [3]:
from ortools.sat.python import cp_model

# Men and Women identifiers
men = ['Richard', 'James', 'John', 'Bill', 'Greg', 'Mario']
women = ['Helen', 'Tracy', 'Linda', 'Sally', 'Wanda', 'Mary']

# Preferences
# Men's preferences
men_preferences = {
    'Richard': [3, 5, 4, 2, 1, 6],
    'James': [3, 2, 5, 6, 4, 1],
    'John': [2, 4, 3, 1, 5, 6],
    'Bill': [5, 6, 4, 2, 3, 1],
    'Greg': [2, 5, 3, 6, 4, 1],
    'Mario': [1, 3, 4, 5, 6, 2]
}

# Women's preferences
women_preferences = {
    'Helen': [2, 4, 5, 3, 6, 1],
    'Tracy': [3, 5, 4, 2, 1, 6],
    'Linda': [1, 3, 6, 2, 4, 5],
    'Sally': [3, 2, 5, 6, 4, 1],
    'Wanda': [6, 4, 2, 1, 3, 5],
    'Mary': [6, 4, 3, 1, 5, 2]
}

# Model
model = cp_model.CpModel()

# Variables: man_to_woman[man][woman] is 1 if man is paired with woman
man_to_woman = {}
for man in men:
    man_to_woman[man] = {}
    for woman in women:
        man_to_woman[man][woman] = model.NewBoolVar(f'{man}_{woman}')

# Each man is paired with exactly one woman
for man in men:
    model.Add(sum(man_to_woman[man][woman] for woman in women) == 1)

# Each woman is paired with exactly one man
for woman in women:
    model.Add(sum(man_to_woman[man][woman] for man in men) == 1)

# Stability constraint: prevent instability in any pairing
for m1 in men:
    for w1 in women:
        for m2 in men:
            for w2 in women:
                if m1 != m2 and w1 != w2:
                    # (m1 prefers w2 over w1) and (w2 prefers m1 over m2)
                    condition1 = men_preferences[m1][women.index(w2)] < men_preferences[m1][women.index(w1)]
                    condition2 = women_preferences[w2][men.index(m1)] < women_preferences[w2][men.index(m2)]
                    # Add constraint to prevent both conditions from being true
                    model.AddBoolOr([man_to_woman[m1][w1].Not(), man_to_woman[m2][w2].Not(), man_to_woman[m1][w2].Not(), man_to_woman[m2][w1].Not()])

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output results
if status == cp_model.OPTIMAL:
    for man in men:
        for woman in women:
            if solver.Value(man_to_woman[man][woman]) == 1:
                print(f'{man} is married to {woman}')
else:
    print("No stable marriage configuration found.")


Richard is married to Linda
James is married to Tracy
John is married to Helen
Bill is married to Wanda
Greg is married to Sally
Mario is married to Mary
